In [ ]:
# Cell 1: imports and constants

import os
import json
import numpy as np
import meshio
import matplotlib.pyplot as plt

import safeincave as sf
import safeincave.PostProcessingTools as post

from matplotlib.gridspec import GridSpec

# For interactive slider in Jupyter
from ipywidgets import IntSlider, interact

# geometry selector if you want it later
GEOMETRY_TYPE = "irregular"  # "regular" / "irregular" / "tilted"

hour = 60 * 60
day = 24 * hour
MPa = 1e6


In [ ]:
# Cell 2: layout + theme

def apply_grey_theme(fig, axes, transparent=True, grid_color="0.92", back_color='0.85'):
    fig.patch.set_facecolor("#212121ff")
    if transparent:
        fig.patch.set_alpha(0.0)
    for ax in axes:
        if ax is not None:
            ax.grid(True, color=grid_color)
            ax.set_axisbelow(True)
            for side in ['bottom', 'top', 'right', 'left']:
                ax.spines[side].set_color('black')
            ax.tick_params(axis='x', colors='black', which='both')
            ax.tick_params(axis='y', colors='black', which='both')
            ax.yaxis.label.set_color('black')
            ax.xaxis.label.set_color('black')
            ax.set_facecolor(back_color)

def create_layout():
    fig = plt.figure(figsize=(16, 9))
    fig.subplots_adjust(top=0.975, bottom=0.120, left=0.060, right=0.986,
                        hspace=0.44, wspace=0.64)
    gs = GridSpec(18, 19, figure=fig)

    ax_logo   = fig.add_subplot(gs[0:2, 0:4])
    ax_info_1 = fig.add_subplot(gs[0:2, 5:9])
    ax_info_2 = fig.add_subplot(gs[0:2, 10:14])
    ax_info_3 = fig.add_subplot(gs[0:2, 15:])

    ax0  = fig.add_subplot(gs[3:12, 0:4])
    ax00 = fig.add_subplot(gs[3:7, 5:9])
    ax01 = fig.add_subplot(gs[3:7, 10:14])
    ax02 = fig.add_subplot(gs[3:7, 15:])
    ax10 = fig.add_subplot(gs[8:12, 5:9])
    ax11 = fig.add_subplot(gs[8:12, 10:14])
    ax12 = fig.add_subplot(gs[8:12, 15:])

    ax30 = fig.add_subplot(gs[14:, 0:5])
    ax31 = fig.add_subplot(gs[14:, 6:12])
    ax32 = fig.add_subplot(gs[14:, 13:19])

    apply_grey_theme(fig, [ax0, ax00, ax01, ax02, ax10, ax11, ax12,
                           ax30, ax31, ax32], transparent=True)

    return (fig, ax_logo, ax_info_1, ax_info_2, ax_info_3,
            ax0, ax00, ax01, ax02, ax10, ax11, ax12, ax30, ax31, ax32)


In [ ]:
# Cell 3: wall profile, subsidence, stress paths

class WallProfileData:
    def __init__(self, output_folder, scale=1.0):
        # u: (nt, n_nodes, 3)
        points, self.time_list, u_field = post.read_node_vector(
            os.path.join(output_folder, "u", "u.xdmf")
        )

        # Read msh node coordinates from mesh
        reader_msh = meshio.read(os.path.join(output_folder, "mesh", "geom.msh"))
        points_msh = reader_msh.points

        # Find "line" entities (cavern wall)
        wall_idx = None
        if hasattr(reader_msh, 'cells_dict'):
            wall_idx = np.unique(reader_msh.cells_dict["line"].flatten())
        elif isinstance(reader_msh.cells, dict):
            if "line" in reader_msh.cells:
                wall_idx = np.unique(reader_msh.cells["line"].flatten())
        else:
            for cell_block in reader_msh.cells:
                if hasattr(cell_block, 'type') and cell_block.type == "line":
                    wall_idx = np.unique(cell_block.data.flatten())
                    break

        if wall_idx is None:
            raise ValueError("No 'line' cells found in mesh.")

        # Map Gmsh → XDMF indexing
        mapping = post.build_mapping(points_msh, points)
        wall_idx = np.array([mapping[i] for i in wall_idx], dtype=int)

        self.wall_points = points[wall_idx]           # (n_wall, 3)
        self.wall_u      = u_field[:, wall_idx, :]    # (nt, n_wall, 3)

        # sort by z
        sorted_idx = np.argsort(self.wall_points[:, 2])
        self.wall_points = self.wall_points[sorted_idx]
        self.wall_u      = self.wall_u[:, sorted_idx, :]

        self.scale = scale
        self.compute_volumes()

    def get_wall_coordinates(self, time_step: int):
        return self.wall_points + self.scale * self.wall_u[time_step, :, :]

    def compute_volumes(self):
        wall_t0 = self.get_wall_coordinates(time_step=0)
        vol_0 = self._trapezoidal_volume(wall_t0[:, 2], wall_t0[:, 0])
        self.volumes = []
        for t_step in range(len(self.time_list)):
            wall_t = self.get_wall_coordinates(time_step=t_step)
            vol = self._trapezoidal_volume(wall_t[:, 2], wall_t[:, 0])
            self.volumes.append(100 * abs(vol_0 - vol) / vol_0)

    @staticmethod
    def _trapezoidal_volume(x, y):
        # solid of revolution around y=0 using trapezoidal rule
        volume = 0.0
        n = len(x)
        for i in range(1, n):
            R = 0.5 * (y[i] + y[i-1])
            A = np.pi * R**2
            d = x[i] - x[i-1]
            volume += A * d
        return volume


class SubsidenceData:
    def __init__(self, output_folder):
        points, self.time_list, u_field = post.read_node_vector(
            os.path.join(output_folder, "u", "u.xdmf")
        )
        # top point approx at (0,0,660)
        top_idx = post.find_closest_point([0, 0, 660], points)
        uz = u_field[:, top_idx, 2]
        # positive subsidence = downward
        self.subsidence = uz - uz[0]


class StressPath:
    def __init__(self, output_folder, probes):
        self.probes = probes
        self.points, self.time_list, self.p_elems = post.read_cell_scalar(
            os.path.join(output_folder, "p_elems", "p_elems.xdmf")
        )
        _, _, self.q_elems = post.read_cell_scalar(
            os.path.join(output_folder, "q_elems", "q_elems.xdmf")
        )

        self.p_probes = np.zeros((probes.shape[0], self.time_list.size))
        self.q_probes = np.zeros((probes.shape[0], self.time_list.size))
        for i, probe in enumerate(probes):
            idx = post.find_closest_point(probe, self.points)
            self.p_probes[i, :] = -self.p_elems[:, idx] / MPa
            self.q_probes[i, :] =  self.q_elems[:, idx] / MPa


In [ ]:
# Cell 4: probes + plotting primitives

def auto_generate_probes(wall_profile: WallProfileData, n_bend_probes: int):
    pts = wall_profile.wall_points
    x = pts[:, 0]
    z = pts[:, 2]

    # top, mid, bottom
    idx_bottom = np.argmin(z)
    idx_top    = np.argmax(z)
    z_mid      = 0.5 * (z[idx_bottom] + z[idx_top])
    idx_mid    = np.argmin(np.abs(z - z_mid))

    probes_idx = [idx_top, idx_mid, idx_bottom]

    dx  = np.gradient(x)
    dz_ = np.gradient(z)
    ddx = np.gradient(dx)
    ddz = np.gradient(dz_)

    denom = (dx*dx + dz_*dz_)**1.5
    denom[denom == 0.0] = np.inf
    curvature = np.abs(dx * ddz - dz_ * ddx) / denom
    curvature[np.isnan(curvature)] = 0.0

    idx_all = np.argsort(curvature)[::-1]

    def too_close(new_i, existing_idx, min_gap=5):
        return any(abs(new_i - ei) < min_gap for ei in existing_idx)

    for i in idx_all:
        if len(probes_idx) >= 3 + n_bend_probes:
            break
        if i in probes_idx:
            continue
        if too_close(i, probes_idx):
            continue
        probes_idx.append(i)

    probes_idx = sorted(probes_idx, key=lambda k: z[k], reverse=True)
    return pts[probes_idx]


def plot_cavern_shape(ax, wall: WallProfileData, t_step: int):
    wall_t0 = wall.get_wall_coordinates(time_step=0)
    wall_tf = wall.get_wall_coordinates(time_step=t_step)
    ax.plot(wall_t0[:, 0], wall_t0[:, 2], "-", color="black", label="Initial shape")
    ax.plot(wall_tf[:, 0], wall_tf[:, 2], "-", color="steelblue", label="Current shape")
    ax.set_xlabel("x (m)")
    ax.set_ylabel("z (m)")
    ax.legend(loc=1, prop={"size": 8})
    ax.axis("equal")


def plot_convergence(ax, wall: WallProfileData, t_step: int):
    ax.plot(wall.time_list/hour, wall.volumes, "-", color="#377eb8", linewidth=2.0)
    ax.scatter(wall.time_list[t_step]/hour, wall.volumes[t_step],
               c="white", edgecolors="black", zorder=10000)
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("Convergence (\%)")


def plot_subsidence(ax, sub: SubsidenceData, t_step: int):
    ax.plot(sub.time_list/hour, 100*sub.subsidence, "-", color="steelblue")
    ax.scatter(sub.time_list[t_step]/hour, 100*sub.subsidence[t_step],
               c="white", edgecolors="black", zorder=10000)
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("Subsidence (cm)")


In [ ]:
# Cell 5: pressure schedule, RD boundary, stress path plots

def plot_pressure_schedule(ax, output_folder, t_step=None):
    pressure_file = os.path.join(output_folder, "pressure_schedule.json")
    if not os.path.exists(pressure_file):
        ax.text(0.5, 0.5, "pressure_schedule.json not found",
                ha='center', va='center', transform=ax.transAxes)
        return None, None
    with open(pressure_file, 'r') as f:
        data = json.load(f)
    t_values = np.array(data["t_values"]) / hour
    p_values = np.array(data["p_values"]) / MPa
    ax.plot(t_values, p_values, "-o", color="darkred", linewidth=2.0,
            markersize=4, label="Pressure schedule")
    if t_step is not None:
        # map time index of fields to closest pressure time
        t_curr = t_values[min(t_step, len(t_values)-1)]
        p_curr = p_values[min(t_step, len(p_values)-1)]
        ax.scatter(t_curr, p_curr, c="white", edgecolors="black", zorder=10000)
    ax.set_xlabel("Time (hours)")
    ax.set_ylabel("Cavern pressure (MPa)")
    ax.legend(loc="best", prop={"size": 8})
    return t_values, p_values


def plot_dilatancy_boundary(ax,
                            D1=0.683, D2=0.512, m=0.75, T0=1.5,
                            sigma_ref=1.0, p_min=0.01, p_max=120.0, npts=400):
    p = np.linspace(p_min, p_max, npts)
    I1 = 3.0 * p

    def q_from_I1(I1_MPa, psi_rad):
        sgn = np.sign(I1_MPa)
        sgn[sgn == 0.0] = 1.0
        denom = (np.sqrt(3.0) * np.cos(psi_rad) - D2 * np.sin(psi_rad))
        sqrtJ2 = D1 * ((I1_MPa / (sgn * sigma_ref)) ** m) / denom + T0
        return np.sqrt(3.0) * sqrtJ2

    psi_c = -np.pi/6.0
    psi_e =  np.pi/6.0
    q_c = q_from_I1(I1, psi_c)
    q_e = q_from_I1(I1, psi_e)

    ax.plot(p, q_c, "-", color="gold", linewidth=1.5, alpha=0.9, label="RD – compression")
    ax.plot(p, q_e, "-", color="turquoise", linewidth=1.5, alpha=0.9, label="RD – extension")
    if not ax.get_legend():
        ax.legend(loc="best", fontsize=8, frameon=True, fancybox=True)


COLORS = ["deepskyblue", "tomato", "orange", "steelblue", "purple", "magenta"]

def plot_stress_paths(ax, stress_path: StressPath, i: int, t_step: int):
    ax.plot(stress_path.p_probes[i], stress_path.q_probes[i],
            "-", color=COLORS[i], linewidth=2.0)
    ax.scatter(stress_path.p_probes[i, t_step], stress_path.q_probes[i, t_step],
               c="white", edgecolors="black", zorder=10000)
    plot_dilatancy_boundary(ax)
    ax.set_xlabel("Mean stress (MPa)", size=10)
    ax.set_ylabel("Von Mises stress (MPa)", size=10)


def plot_probes(ax, probes):
    for i, probe in enumerate(probes):
        ax.scatter(probe[0], probe[2], c=COLORS[i % len(COLORS)],
                   edgecolors="black", zorder=10000, s=80)


In [ ]:
# Cell 6: metadata + one function that draws the full figure for a time step

def show_metadata(ax_mesh, ax_model, ax_solver, output_folder):
    log_file = os.path.join(output_folder, "log.txt")
    if not os.path.exists(log_file):
        for ax in (ax_mesh, ax_model, ax_solver):
            ax.axis("off")
        return
    with open(log_file) as f:
        log = f.readlines()

    dh = 0.3
    h = 0.8
    for i, line in enumerate(log):
        if "| Mesh info:" in line:
            line_split = log[i+4].split("|")
            n_elems = int(line_split[1])
            n_nodes = int(line_split[2])
            location = line_split[3].strip(" ")
            ax_mesh.text(0, h-0*dh, "Mesh info:", size=12)
            ax_mesh.text(0, h-1*dh, f"- Location: {location}", size=10)
            ax_mesh.text(0, h-2*dh, f"- Elements: {n_elems}", size=10)
            ax_mesh.text(0, h-3*dh, f"- Nodes: {n_nodes}", size=10)
            ax_mesh.axis('off')
        elif "| Solver info:" in line:
            line_split = log[i+4].split("|")
            ksp_solver = line_split[1].strip(" ")
            ksp_pc = line_split[2].strip(" ")
            tol = float(line_split[3])
            max_ite = int(line_split[4])
            cpu_time = log[-1].strip(" ").strip("Total time: ")[:8] if len(log) > 0 else "N/A"
            ax_solver.text(0, h-0*dh, "Simulation info:", size=12)
            ax_solver.text(0, h-1*dh, f"- Solver: {ksp_solver}, PC: {ksp_pc}", size=10)
            ax_solver.text(0, h-2*dh, f"- tol: {tol}, max it: {max_ite}", size=10)
            ax_solver.text(0, h-3*dh, f"- CPU Time: {cpu_time}", size=10)
            ax_solver.axis('off')
        elif "| Constitutive model:" in line:
            elastic_elems = log[i+4].split("|")[2].strip(" ") if i+4 < len(log) else "N/A"
            nonelastic_elems = log[i+5].split("|")[2].strip(" ") if i+5 < len(log) else "N/A"
            thermoelastic_elems = log[i+6].split("|")[2].strip(" ") if i+6 < len(log) else "N/A"
            ax_model.text(0, h-0*dh, "Constitutive model:", size=12)
            ax_model.text(0, h-1*dh, f"- Elastic: {elastic_elems}", size=10)
            ax_model.text(0, h-2*dh, f"- Non-elastic: {nonelastic_elems}", size=10)
            ax_model.text(0, h-3*dh, f"- Thermoelastic: {thermoelastic_elems}", size=10)
            ax_model.axis('off')


def plot_state(t_step, wall_profile, subsidence_data, stress_path,
               probes, output_folder, info_axes, stress_axes, ax0, ax30, ax31, ax32, fig):
    # clear stress axes
    for ax in stress_axes:
        ax.cla()
    ax0.cla(); ax30.cla(); ax31.cla(); ax32.cla()

    ax_mesh, ax_model, ax_solver = info_axes

    # mesh/model/solver boxes (static)
    ax_mesh.cla(); ax_model.cla(); ax_solver.cla()
    show_metadata(ax_mesh, ax_model, ax_solver, os.path.join(output_folder, "operation"))

    # cavern shape
    plot_cavern_shape(ax0, wall_profile, t_step)
    plot_probes(ax0, probes)

    # convergence & subsidence
    plot_convergence(ax31, wall_profile, t_step)
    plot_subsidence(ax32, subsidence_data, t_step)

    # pressure schedule
    plot_pressure_schedule(ax30, output_folder, t_step=t_step)

    # stress paths for as many probes as we have axes
    n_axes = len(stress_axes)
    n_probes = probes.shape[0]
    n_use = min(n_axes, n_probes)
    for i in range(n_use):
        plot_stress_paths(stress_axes[i], stress_path, i, t_step)
    for j in range(n_use, n_axes):
        stress_axes[j].axis("off")

    apply_grey_theme(fig, [ax0, *stress_axes, ax30, ax31, ax32], transparent=True)
    fig.suptitle(f"Time step {t_step}  (t = {wall_profile.time_list[t_step]/hour:.1f} h)", y=0.99)
    plt.show()


In [ ]:
# Cell 7: load data and build interactive viewer

# base folder (same as in your MultipleCycles script)
output_folder = os.path.join("output", "case_sinus2x_10days_regular")
operation_folder = os.path.join(output_folder, "operation")

# load data
wall_profile = WallProfileData(operation_folder, scale=1.0)
subsidence_data = SubsidenceData(operation_folder)

# number of bend probes based on geometry
if GEOMETRY_TYPE == "irregular":
    n_bend_probes = 3
else:
    n_bend_probes = 1

probes = auto_generate_probes(wall_profile, n_bend_probes=n_bend_probes)

# shift mid + bend probes into salt by small offset in x
OFFSET_R = 0.15
n_probes = probes.shape[0]
for k in range(n_probes):
    if 0 < k < n_probes - 1:
        probes[k, 0] += OFFSET_R

stress_path = StressPath(operation_folder, probes)
n_steps = stress_path.time_list.size

# create figure & axes once
(fig, ax_logo, ax_info_1, ax_info_2, ax_info_3,
 ax0, ax00, ax01, ax02, ax10, ax11, ax12,
 ax30, ax31, ax32) = create_layout()

stress_axes = [ax00, ax01, ax02, ax10, ax11, ax12]
info_axes = (ax_info_1, ax_info_2, ax_info_3)

# simple logo placeholder
ax_logo.axis("off")
ax_logo.text(0.5, 0.5, "SafeInCave\nviewer", ha="center", va="center", fontsize=14)

# interactive slider
slider = IntSlider(min=0, max=n_steps-1, step=1, value=0,
                   description="Time step", continuous_update=False)

@interact(t_step=slider)
def _update(t_step):
    plot_state(t_step, wall_profile, subsidence_data, stress_path,
               probes, output_folder, info_axes, stress_axes,
               ax0, ax30, ax31, ax32, fig)
